In [1]:
import urllib.request
from bs4 import BeautifulSoup

In [2]:
import re

Ouverture de la page concernant un film

In [3]:
# Génère les url des différentes pages de critiques liées à un film.
def generate_var(hero, i) :
    return ["http://www.senscritique.com/film/"+hero[1]+"/critiques/page-"+str(i), hero[0], i*10]

In [4]:
# Récupère le lien vers la critique d'un film à partir de l'adresse de la page générale consacrée à ce film.
def get_link_critique(link_film) :
    page=urllib.request.urlopen(link_film) 
    strpage=page.read()
    soup = BeautifulSoup(strpage, 'html.parser')

    res=[]
    res2= []
    
    for link in soup.find_all('a'):
        if "critique/" in link.get('href') :
            res.append(link.get('href'))
    for link in set(res) :
        res2.append ("https://www.senscritique.com"+link)
    return res2

In [5]:
# Récupère toutes les pages sur une page de critiques.
#hdr grâce à https://www.whatismybrowser.com/detect/what-http-headers-is-my-browser-sending
def acces_critique(lien_film) :
    ens={}
    i=0
    
    for link in get_link_critique(lien_film) :
        hdr = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.99 Safari/537.36', 
               'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8', 
               'User-Agent':'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.99 Safari/537.36', 
              'accept_language' : 'fr-FR,fr;q=0.8,en-US;q=0.6,en;q=0.4',
              'connection' : 'keep-alive'}
        page=urllib.request.urlopen(link, timeout=1.5, headers=hdr) 
        strpage=page.read()
        ens[i]= BeautifulSoup(strpage, 'html.parser')
        i+=1
    return ens

In [6]:
# Ne conserve que les critiques.
def get_review(soup) :
    return soup.find_all("div", class_="rvi-review-content")

In [7]:
# Ne conserve que la note donnnée par la critique.
def get_note (page) :
    valeurs=page.find_all("span", itemprop="ratingValue")
    val=""
    for el in valeurs :
        val+=el.get_text()
    return re.sub("[^0-9]", "", val)

In [8]:
# Retire d'éventuelles publicités.
def delete_adcontent(divs) :
    pollution = []
    for i in divs :
        pollution.extend(i.findAll("div", { "class" : "rvi-adContent right" }))
    if len(pollution) > 0 :
        pol=pollution[0]
        pol.extract()

In [9]:
# Ne garde que le texte.
def get_content(divs) :
    content=""
    for el in divs :
        content+=el.get_text()
    return content

In [10]:
# Supprime les tabulations et sauts de ligne.
def delete_tab_new_line (contenu) :
    a_sup = ["\t", "\n"]
    for sup in a_sup : 
        contenu=re.sub(sup, "", contenu)
    return contenu

In [11]:
# Combine get_content et delete_tab_new_line pour ne conserver que la critique et sous la forme souhaitée.
def get_clear_content(div) :
    return delete_tab_new_line(get_content(div))

In [12]:
# Fonction finale, qui combine toutes les précédentes pour extraire les critiques liées à un film.
def extraire_avis(link, nom, ab=0) :
    critiques=acces_critique(link)
    for i in range (len(critiques)) :
        critique= critiques[i]
        mydivs=get_review(critique)
        note=get_note(critique)
        delete_adcontent(mydivs)
        final=get_clear_content(mydivs)
        final.encode('utf-8', 'ignore')
        enrichi=str(note)+'\t'+final
        f = open(str(note)+nom+str(ab+i)+'.txt', 'w', errors='ignore')
        f.write(enrichi)
        f.close()
    return "Bravo bravo ! Ca a l'air d'avoir fonctionné !"

In [13]:
# Etape préalable : génération des adresses auxquelles récupérer les critiques.
marvel1 = [["IM", "Iron_Man/451223"], ["AM", "Ant_Man/445917"], 
           ["Th", "Thor_Le_Monde_des_tenebres/1358693"]]
marvel2 = [["Av", "Avengers/493660"], ["CA", "Captain_America_Civil_War/11040752"]]
marvel3 = [["DS", "Doctor_Strange/392811"], ["GG", "Les_Gardiens_de_la_galaxie/406937"],
           ["IH", "L_Incroyable_Hulk/437716"]]
marvel4 = [["XD", "X_Men_Days_of_Future_Past/422063"], ["XC", "X_Men_Le_Commencement/487010"], 
           ["CS", "Captain_America_Le_Soldat_de_l_hiver/395521"], ["SM", "Spider_Man/387292"],
          ["AU", "Avengers_L_Ere_d_Ultron/437798"], ["Sb", "Spider_Man_2/434094"], 
          ["XM", "X_Men/446853"], ["DP", "Deadpool/362143"]]
marvel5 = [["CV", "Captain_America_First_Avenger/440653"], ["TT", "Thor_Le_Monde_des_tenebres/1358693"],
          ["AS", "The_Amazing_Spider_Man/483863"], ["St", "Spider_Man_3/417884"], ["It", "Iron_Man_3/385437"]]

a_extraire=[]

for marvel in marvel1 : 
    for i in range (8, 15) :
        a_extraire.append(generate_var(marvel, i))
a_extraire[0]

['http://www.senscritique.com/film/Iron_Man/451223/critiques/page-8', 'IM', 80]

In [14]:
print(a_extraire[i][0])

http://www.senscritique.com/film/Thor_Le_Monde_des_tenebres/1358693/critiques/page-8


In [15]:
# Récupération et enregistrement des critiques.
for i in range (len(a_extraire)) :
    extraire_avis(link=a_extraire[i][0], nom=a_extraire[i][1], ab=a_extraire[i][2]-10)
    #except IncompleteRead :
     #   print ("N'a pas fonctionné pour"+str(a_extraire[i][0]))

HTTPError: HTTP Error 403: Forbidden